In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping
from keras.utils import np_utils

In [2]:
#data = pd.read_csv("./input/train.csv")
data = np.load("augmented.npz")

In [3]:
X = data['x']
Y = data['y']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

In [5]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

In [6]:
print("Shape of X_train: {}, shape of y_train: {}".format(X_train.shape, y_train.shape))
print("Shape of X_test: {}, shape of y_test: {}".format(X_test.shape, y_test.shape))

Shape of X_train: (378000, 28, 28, 1), shape of y_train: (378000,)
Shape of X_test: (42000, 28, 28, 1), shape of y_test: (42000,)


In [17]:
X_train[X_train < 0] = 0
X_test[X_test < 0] = 0

In [21]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [22]:
def cnn_model():
    model = Sequential()
    model.add(Conv2D(10, (3,3), input_shape=(28, 28, 1), activation='relu'))
    model.add(Conv2D(20, (5,5), strides=(2,2), activation='relu'))
    model.add(Conv2D(40, (5,5), activation='relu'))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [45]:
model = cnn_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

Train on 37800 samples, validate on 4200 samples
Epoch 1/10
37800/37800 [==============================] - 14s 379us/step - loss: 0.3884 - acc: 0.8882 - val_loss: 0.1417 - val_acc: 0.9548
Epoch 2/10
37800/37800 [==============================] - 9s 237us/step - loss: 0.0900 - acc: 0.9725 - val_loss: 0.0851 - val_acc: 0.9740
Epoch 3/10
37800/37800 [==============================] - 9s 241us/step - loss: 0.0603 - acc: 0.9815 - val_loss: 0.0641 - val_acc: 0.9783
Epoch 4/10
37800/37800 [==============================] - 9s 247us/step - loss: 0.0458 - acc: 0.9859 - val_loss: 0.0607 - val_acc: 0.9824
Epoch 5/10
37800/37800 [==============================] - 9s 246us/step - loss: 0.0364 - acc: 0.9889 - val_loss: 0.0510 - val_acc: 0.9852
Epoch 6/10
37800/37800 [==============================] - 9s 240us/step - loss: 0.0312 - acc: 0.9907 - val_loss: 0.0519 - val_acc: 0.9860
Epoch 7/10
37800/37800 [==============================] - 9s 234us/step - loss: 0.0257 - acc: 0.9916 - val_loss: 0.0455 - 

In [47]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("ConvNet Error: %.2f%%" % (100-scores[1]*100))

ConvNet Error: 1.33%


In [48]:
#added two more fully connected layers

def cnn2_model():
    model = Sequential()
    model.add(Conv2D(10, (3,3), input_shape=(28, 28, 1), activation='relu'))
    model.add(Conv2D(20, (5,5), strides=(2,2), activation='relu'))
    model.add(Conv2D(40, (5,5), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [49]:
model = cnn2_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

Train on 37800 samples, validate on 4200 samples
Epoch 1/10
37800/37800 [==============================] - 10s 268us/step - loss: 0.3366 - acc: 0.8997 - val_loss: 0.1279 - val_acc: 0.9607
Epoch 2/10
37800/37800 [==============================] - 10s 256us/step - loss: 0.0833 - acc: 0.9748 - val_loss: 0.0820 - val_acc: 0.9755
Epoch 3/10
37800/37800 [==============================] - 10s 258us/step - loss: 0.0574 - acc: 0.9816 - val_loss: 0.0730 - val_acc: 0.9798
Epoch 4/10
37800/37800 [==============================] - 10s 255us/step - loss: 0.0406 - acc: 0.9874 - val_loss: 0.0638 - val_acc: 0.9793
Epoch 5/10
37800/37800 [==============================] - 10s 264us/step - loss: 0.0316 - acc: 0.9900 - val_loss: 0.0571 - val_acc: 0.9821
Epoch 6/10
37800/37800 [==============================] - 10s 254us/step - loss: 0.0230 - acc: 0.9926 - val_loss: 0.0524 - val_acc: 0.9843
Epoch 7/10
37800/37800 [==============================] - 10s 253us/step - loss: 0.0181 - acc: 0.9946 - val_loss: 0.0

In [50]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("ConvNet Error: %.2f%%" % (100-scores[1]*100))

ConvNet Error: 1.88%


In [78]:
# keep only one additional fully connected layer, add average pooling 
# instead of stride and dropout before flattening for regularization

def cnn3_model():
    model = Sequential()
    model.add(Conv2D(10, (3,3), input_shape=(28, 28, 1), activation='relu'))
    model.add(Conv2D(20, (5,5), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2)))
    model.add(Conv2D(40, (5,5), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [79]:
model = cnn3_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, callbacks=[EarlyStopping(monitor='val_acc', min_delta=0.001, patience=2, verbose=1, mode='auto')])

Train on 37800 samples, validate on 4200 samples
Epoch 1/10
37800/37800 [==============================] - 25s 661us/step - loss: 0.3269 - acc: 0.9029 - val_loss: 0.1047 - val_acc: 0.9683
Epoch 2/10
37800/37800 [==============================] - 23s 618us/step - loss: 0.0755 - acc: 0.9771 - val_loss: 0.0589 - val_acc: 0.9836
Epoch 3/10
37800/37800 [==============================] - 24s 629us/step - loss: 0.0524 - acc: 0.9839 - val_loss: 0.0552 - val_acc: 0.9819
Epoch 4/10
37800/37800 [==============================] - 23s 612us/step - loss: 0.0373 - acc: 0.9878 - val_loss: 0.0464 - val_acc: 0.9850
Epoch 5/10
37800/37800 [==============================] - 23s 619us/step - loss: 0.0313 - acc: 0.9902 - val_loss: 0.0424 - val_acc: 0.9888
Epoch 6/10
37800/37800 [==============================] - 24s 629us/step - loss: 0.0264 - acc: 0.9919 - val_loss: 0.0445 - val_acc: 0.9888
Epoch 7/10
37800/37800 [==============================] - 23s 615us/step - loss: 0.0216 - acc: 0.9929 - val_loss: 0.0

In [80]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("ConvNet Error: %.2f%%" % (100-scores[1]*100))

ConvNet Error: 1.21%


In [66]:
submdata = pd.read_csv("./input/test.csv")

In [70]:
m = len(submdata)
X_subm = np.array(submdata)
X_subm = X_subm.reshape(m, 28, 28, 1).astype('float32')

In [71]:
pred = model.predict(X_subm)

In [72]:
subm = pd.DataFrame({"ImageId": np.arange(1, m+1), "Label": np.argmax(pred, axis=1)})
subm.to_csv("subm3.csv", index=False)

In [25]:
# keep only one additional fully connected layer, add average pooling 
# instead of stride and dropout before flattening for regularization

def cnn4_model():
    model = Sequential()
    model.add(Conv2D(10, (3,3), input_shape=(28, 28, 1), activation='relu'))
    model.add(Conv2D(20, (5,5), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2)))
    model.add(Conv2D(40, (5,5), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [26]:
model = cnn4_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, callbacks=[EarlyStopping(monitor='val_acc', min_delta=0.001, patience=2, verbose=1, mode='auto')])

Train on 378000 samples, validate on 42000 samples
Epoch 1/10
378000/378000 [==============================] - 27s 71us/step - loss: 0.1790 - acc: 0.9446 - val_loss: 0.0627 - val_acc: 0.9810
Epoch 2/10
378000/378000 [==============================] - 23s 62us/step - loss: 0.0539 - acc: 0.9832 - val_loss: 0.0414 - val_acc: 0.9876
Epoch 3/10
378000/378000 [==============================] - 22s 57us/step - loss: 0.0368 - acc: 0.9883 - val_loss: 0.0296 - val_acc: 0.9903
Epoch 4/10
378000/378000 [==============================] - 21s 56us/step - loss: 0.0281 - acc: 0.9911 - val_loss: 0.0281 - val_acc: 0.9911
Epoch 5/10
378000/378000 [==============================] - 21s 55us/step - loss: 0.0225 - acc: 0.9929 - val_loss: 0.0242 - val_acc: 0.9927
Epoch 6/10
378000/378000 [==============================] - 21s 55us/step - loss: 0.0182 - acc: 0.9940 - val_loss: 0.0231 - val_acc: 0.9929
Epoch 7/10
378000/378000 [==============================] - 21s 56us/step - loss: 0.0156 - acc: 0.9948 - val_

In [27]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("ConvNet Error: %.2f%%" % (100-scores[1]*100))

ConvNet Error: 0.61%


In [29]:
def submfile(fname="subm.csv"):
    submdata = pd.read_csv("./input/test.csv")
    m = len(submdata)
    X_subm = np.array(submdata)
    X_subm = X_subm.reshape(m, 28, 28, 1).astype('float32')
    pred = model.predict(X_subm)
    subm = pd.DataFrame({"ImageId": np.arange(1, m+1), "Label": np.argmax(pred, axis=1)})
    subm.to_csv(fname, index=False)

In [30]:
submfile("subm4.csv")

In [32]:
# keep only one additional fully connected layer, add average pooling 
# instead of stride and dropout before flattening for regularization

# apparently there was some overfitting, try to move the dropout before the softmax layer

def cnn5_model():
    model = Sequential()
    model.add(Conv2D(10, (3,3), input_shape=(28, 28, 1), activation='relu'))
    model.add(Conv2D(20, (5,5), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2)))
    model.add(Conv2D(40, (5,5), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [34]:
model = cnn5_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, callbacks=[EarlyStopping(monitor='val_acc', min_delta=0.001, patience=2, verbose=1, mode='auto')])

Train on 378000 samples, validate on 42000 samples
Epoch 1/10
378000/378000 [==============================] - 22s 57us/step - loss: 0.1917 - acc: 0.9407 - val_loss: 0.0595 - val_acc: 0.9817
Epoch 2/10
378000/378000 [==============================] - 21s 57us/step - loss: 0.0591 - acc: 0.9817 - val_loss: 0.0511 - val_acc: 0.9838
Epoch 3/10
378000/378000 [==============================] - 21s 55us/step - loss: 0.0420 - acc: 0.9871 - val_loss: 0.0339 - val_acc: 0.9894
Epoch 4/10
378000/378000 [==============================] - 20s 54us/step - loss: 0.0326 - acc: 0.9898 - val_loss: 0.0346 - val_acc: 0.9891
Epoch 5/10
378000/378000 [==============================] - 20s 54us/step - loss: 0.0259 - acc: 0.9917 - val_loss: 0.0264 - val_acc: 0.9918
Epoch 6/10
378000/378000 [==============================] - 20s 54us/step - loss: 0.0220 - acc: 0.9929 - val_loss: 0.0245 - val_acc: 0.9927
Epoch 7/10
378000/378000 [==============================] - 20s 54us/step - loss: 0.0194 - acc: 0.9937 - val_

In [35]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("ConvNet Error: %.2f%%" % (100-scores[1]*100))

ConvNet Error: 0.71%


In [38]:
# keep only one additional fully connected layer, add average pooling 
# instead of stride and dropout before flattening for regularization

# apparently there was some overfitting, try to move the dropout before the softmax layer

def cnn6_model():
    model = Sequential()
    model.add(Conv2D(10, (3,3), input_shape=(28, 28, 1), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(20, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(40, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [41]:
model = cnn6_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=200, callbacks=[EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=2, verbose=1, mode='auto')])

Train on 378000 samples, validate on 42000 samples
Epoch 1/30
378000/378000 [==============================] - 17s 44us/step - loss: 0.2602 - acc: 0.9175 - val_loss: 0.0952 - val_acc: 0.9705
Epoch 2/30
378000/378000 [==============================] - 16s 43us/step - loss: 0.0894 - acc: 0.9721 - val_loss: 0.0609 - val_acc: 0.9814
Epoch 3/30
378000/378000 [==============================] - 16s 43us/step - loss: 0.0651 - acc: 0.9797 - val_loss: 0.0554 - val_acc: 0.9830
Epoch 4/30
378000/378000 [==============================] - 16s 43us/step - loss: 0.0535 - acc: 0.9833 - val_loss: 0.0432 - val_acc: 0.9868
Epoch 5/30
378000/378000 [==============================] - 16s 43us/step - loss: 0.0458 - acc: 0.9858 - val_loss: 0.0419 - val_acc: 0.9876
Epoch 6/30
378000/378000 [==============================] - 16s 43us/step - loss: 0.0402 - acc: 0.9874 - val_loss: 0.0376 - val_acc: 0.9883
Epoch 7/30
378000/378000 [==============================] - 16s 42us/step - loss: 0.0361 - acc: 0.9885 - val_

In [43]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("ConvNet Error: %.2f%%" % (100-scores[1]*100))

ConvNet Error: 0.87%


In [44]:
submfile("subm6.csv")